<a href="https://colab.research.google.com/github/surajsrivathsa/NL2SQL/blob/main/GAZP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import traceback, torch
import os, sys, glob, tqdm
import random, json, re
import argparse, itertools
from collections import defaultdict, Counter
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
gazp_basepath = "/content/drive/My Drive/NL2SQL/Models/gazp"

In [ ]:
#!mkdir "/content/drive/My Drive/NL2SQL/Models/gazp/cache/bert"

In [ ]:
# !wget https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt -O "/content/drive/My Drive/NL2SQL/Models/gazp/cache/bert/vocab.txt"
# !wget https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-pytorch_model.bin -O "/content/drive/My Drive/NL2SQL/Models/gazp/cache/bert/pytorch_model.bin"
# !wget https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json -O "/content/drive/My Drive/NL2SQL/Models/gazp/cache/bert/config.json"

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
sys.path.append(gazp_basepath)

In [ ]:
!pip install pysqlite3
!pip install embeddings
!pip install vocab
!pip install transformers
!pip install loguru
# !pip install converter

     |████████████████████████████████| 40kB 5.4MB/s 
  Created wheel for pysqlite3: filename=pysqlite3-0.4.6-cp37-cp37m-linux_x86_64.whl size=132572 sha256=326ba84a3e00b0d08645973e83b65cbfb15e1d404a95a991d60b3a9ad22d9d82
  Stored in directory: /root/.cache/pip/wheels/52/bd/85/9f859f07cb153f4d7e3ed094b821fb08b85935c2522c3bd9d4
Successfully built pysqlite3
     |████████████████████████████████| 2.3MB 7.2MB/s 
     |████████████████████████████████| 901kB 38.4MB/s 
     |████████████████████████████████| 3.3MB 47.1MB/s 
     |████████████████████████████████| 61kB 5.4MB/s 


In [ ]:
import sqlite3, converter, embeddings as E, converter, vocab as Vocab, transformers

In [ ]:
!ls -ltr "/content/drive/My Drive/NL2SQL/Models/gazp"

total 224
-rw------- 1 root root  2967 Feb  1 12:49 utils.py
-rw------- 1 root root  3378 Feb  1 12:49 timed_execute.py
-rw------- 1 root root  4830 Feb  1 12:49 README.md
-rw------- 1 root root  8291 Feb  1 12:49 preprocess_sql2nl_sparc.py
-rw------- 1 root root  9725 Feb  1 12:49 preprocess_nl2sql_sparc.py
-rw------- 1 root root 14555 Feb  1 12:49 preprocess_nl2sql_novalue.py
-rw------- 1 root root  9706 Feb  1 12:49 preprocess_nl2sql_cosql.py
-rw------- 1 root root   967 Feb  1 12:49 pred.sh
-rw------- 1 root root  3435 Feb  1 12:49 predict_sparc.py
-rw------- 1 root root  3652 Feb  1 12:49 predict.py
-rw------- 1 root root  1069 Feb  1 12:49 LICENSE
-rw------- 1 root root 17672 Feb  1 12:49 editsql_postprocess.py
-rw------- 1 root root   379 Feb  1 12:49 download.sh
-rw------- 1 root root   234 Feb  1 12:49 download_nltk.py
-rw------- 1 root root  4083 Feb  1 12:49 dataset.py
-rw------- 1 root root 28125 Feb  1 12:49 bleu.py
drwx------ 2 root root  4096 May 10 12:35 model
drwx-----

In [ ]:
%cd "/content/drive/My Drive/NL2SQL/Models/gazp"

/content/drive/My Drive/NL2SQL/Models/gazp


In [ ]:
!pwd

/content/drive/My Drive/NL2SQL/Models/gazp


In [ ]:
os.path.abspath(os.getcwd())

'/content/drive/My Drive/NL2SQL/Models/gazp'

# New Section

In [ ]:
!python train.py --test_only --dataset cosql --drnn 300 --keep_values --model nl2sql --resume "/content/drive/My Drive/NL2SQL/Models/gazp/exp/nl2sql/r1/cosql_nl2sql.tar"

2021-06-06 14:57:07.617522: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
cache/data_nl2sql_cosql.pt
pruned length from 6459 to 6215
/content/drive/My Drive/NL2SQL/Models/gazp/data/database
/content/drive/My Drive/NL2SQL/Models/gazp/data/spider/tables.json
/content/drive/My Drive/NL2SQL/Models/gazp/data/database
/content/drive/My Drive/NL2SQL/Models/gazp/data/spider/tables.json
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1631: FutureWarning: Calling DistilBertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  FutureWarning,
Some weights of the model checkpoint at cache/bert were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_pr

In [ ]:
#!mkdir gen
!python generate.py --dataset cosql --num 1000  --resume "/content/drive/My Drive/NL2SQL/Models/gazp/exp/sql2nl/r1/cosql_sql2nl.tar" --fout "/content/drive/My Drive/NL2SQL/Models/gazp/gen/gen_pretrained.json"  --fparser "/content/drive/My Drive/NL2SQL/Models/gazp/exp/nl2sql/r1/cosql_nl2sql.tar" --ftrain "/content/drive/My Drive/NL2SQL/Models/gazp/data/cosql/train.json"

Streaming output truncated to the last 5000 lines.
db: /content/drive/My Drive/NL2SQL/Models/gazp/data/database/voter_1/voter_1.sqlite
/content/drive/My Drive/NL2SQL/Models/gazp/data/database/voter_1/voter_1.sqlite
db: /content/drive/My Drive/NL2SQL/Models/gazp/data/database/voter_1/voter_1.sqlite
official eval:  69% 2707/3942 [13:35<05:58,  3.45it/s]/content/drive/My Drive/NL2SQL/Models/gazp/data/database/dog_kennels/dog_kennels.sqlite
db: /content/drive/My Drive/NL2SQL/Models/gazp/data/database/dog_kennels/dog_kennels.sqlite
/content/drive/My Drive/NL2SQL/Models/gazp/data/database/dog_kennels/dog_kennels.sqlite
db: /content/drive/My Drive/NL2SQL/Models/gazp/data/database/dog_kennels/dog_kennels.sqlite
official eval:  69% 2708/3942 [13:36<06:08,  3.35it/s]/content/drive/My Drive/NL2SQL/Models/gazp/data/database/pets_1/pets_1.sqlite
db: /content/drive/My Drive/NL2SQL/Models/gazp/data/database/pets_1/pets_1.sqlite
/content/drive/My Drive/NL2SQL/Models/gazp/data/database/pets_1/pets_1.sq

In [ ]:
sqltnsr = torch.load("/content/drive/My Drive/NL2SQL/Models/gazp/gen/gen_pretrained.sql.pt")
questiontnsr = torch.load("/content/drive/My Drive/NL2SQL/Models/gazp/gen/gen_pretrained.ques.pt")
alltnsr = torch.load("/content/drive/My Drive/NL2SQL/Models/gazp/gen/gen_pretrained.pt")

In [ ]:
for x in sqltnsr:
  print(x)
  break;

{'id': 'gen:0', 'columns': [{'oname': '*', 'name': '*', 'type': 'all', 'key': 'battle.*', 'table_name': 'battle'}, {'oname': 'id', 'name': 'id', 'type': 'key', 'key': 'battle.id', 'table_name': 'battle'}, {'oname': 'name', 'name': 'name', 'type': 'text', 'key': 'battle.name', 'table_name': 'battle'}, {'oname': 'date', 'name': 'date', 'type': 'text', 'key': 'battle.date', 'table_name': 'battle'}, {'oname': 'bulgarian_commander', 'name': 'bulgarian commander', 'type': 'text', 'key': 'battle.bulgarian_commander', 'table_name': 'battle'}, {'oname': 'latin_commander', 'name': 'latin commander', 'type': 'text', 'key': 'battle.latin_commander', 'table_name': 'battle'}, {'oname': 'result', 'name': 'result', 'type': 'text', 'key': 'battle.result', 'table_name': 'battle'}, {'oname': '*', 'name': '*', 'type': 'all', 'key': 'ship.*', 'table_name': 'ship'}, {'oname': 'lost_in_battle', 'name': 'lost in battle', 'type': 'key', 'key': 'ship.lost_in_battle', 'table_name': 'ship'}, {'oname': 'id', 'name

In [ ]:
for x in questiontnsr:
  print(x)
  break;

{'id': 'gen:0', 'columns': [{'oname': '*', 'name': '*', 'type': 'all', 'key': 'battle.*', 'table_name': 'battle'}, {'oname': 'id', 'name': 'id', 'type': 'key', 'key': 'battle.id', 'table_name': 'battle'}, {'oname': 'name', 'name': 'name', 'type': 'text', 'key': 'battle.name', 'table_name': 'battle'}, {'oname': 'date', 'name': 'date', 'type': 'text', 'key': 'battle.date', 'table_name': 'battle'}, {'oname': 'bulgarian_commander', 'name': 'bulgarian commander', 'type': 'text', 'key': 'battle.bulgarian_commander', 'table_name': 'battle'}, {'oname': 'latin_commander', 'name': 'latin commander', 'type': 'text', 'key': 'battle.latin_commander', 'table_name': 'battle'}, {'oname': 'result', 'name': 'result', 'type': 'text', 'key': 'battle.result', 'table_name': 'battle'}, {'oname': '*', 'name': '*', 'type': 'all', 'key': 'ship.*', 'table_name': 'ship'}, {'oname': 'lost_in_battle', 'name': 'lost in battle', 'type': 'key', 'key': 'ship.lost_in_battle', 'table_name': 'ship'}, {'oname': 'id', 'name

In [ ]:
for x in alltnsr:
  print(x)
  break;

{'id': 'gen:7', 'columns': [{'oname': '*', 'name': '*', 'type': 'all', 'key': 'employee.*', 'table_name': 'employee'}, {'oname': 'Employee_ID', 'name': 'employee id', 'type': 'key', 'key': 'employee.employee_id', 'table_name': 'employee'}, {'oname': 'Name', 'name': 'name', 'type': 'text', 'key': 'employee.name', 'table_name': 'employee'}, {'oname': 'Age', 'name': 'age', 'type': 'number', 'key': 'employee.age', 'table_name': 'employee'}, {'oname': 'City', 'name': 'city', 'type': 'text', 'key': 'employee.city', 'table_name': 'employee'}, {'oname': '*', 'name': '*', 'type': 'all', 'key': 'shop.*', 'table_name': 'shop'}, {'oname': 'Shop_ID', 'name': 'shop id', 'type': 'key', 'key': 'shop.shop_id', 'table_name': 'shop'}, {'oname': 'Name', 'name': 'name', 'type': 'text', 'key': 'shop.name', 'table_name': 'shop'}, {'oname': 'Location', 'name': 'location', 'type': 'text', 'key': 'shop.location', 'table_name': 'shop'}, {'oname': 'District', 'name': 'district', 'type': 'text', 'key': 'shop.distr

# New Section

In [ ]:
!python preprocess_nl2sql.py

In [ ]:
!python preprocess_sql2nl.py

In [ ]:
!pwd

In [ ]:
!python train.py --dataset spider --name r1 --keep_values --model nl2sql

In [ ]:
!python train.py --dataset spider --name r1 --keep_values --model nl2sql

In [ ]:
!python train.py --dataset cosql --name r1 --model sql2nl --fparser "/content/drive/My Drive/NL2SQL/Models/gazp/exp/sql2nl/r1/best.tar" --fcache "/content/drive/My Drive/NL2SQL/Models/gazp/cache/data_sql2nl_sparc_cosql.pt" 

In [ ]:
#conn = sqlite3.connect("/content/drive/My Drive/NL2SQL/Models/gazp/data/database/department_management/department_management.sqlite")

In [ ]:
#conn.close()

In [ ]:
!python preprocess_nl2sql_cosql.py

In [ ]:
!python preprocess_sql2nl_cosql.py

In [ ]:
#!mkdir gen
!python generate.py --dataset cosql --num 1000  --resume "/content/drive/My Drive/NL2SQL/Models/gazp/exp/sql2nl/r1/best.tar" --fout "/content/drive/My Drive/NL2SQL/Models/gazp/gen/gen1.json"  --fparser "/content/drive/My Drive/NL2SQL/Models/gazp/exp/nl2sql/r1/best.tar"

2021-06-06 16:02:04.568889: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Namespace(aug='ans2sql', batch=20, beam_size=0, dataset='cosql', db='/content/drive/My Drive/NL2SQL/Models/gazp/data/database', db_split='eval', dcache='cache/bert', debug=False, dexp='exp', epoch=50, fout='/content/drive/My Drive/NL2SQL/Models/gazp/gen/gen1.json', fparser='/content/drive/My Drive/NL2SQL/Models/gazp/exp/nl2sql/r1/best.tar', ftrain='/content/drive/My Drive/NL2SQL/Models/gazp/data/spider/train.json', gpu=True, lr=5e-05, max_grad_norm=20, max_query_len=50, max_value_len=20, model='sql2nl', n_proc=1, name='default', num_gen=1000, restart_optim=False, restart_scheduler=False, resume='/content/drive/My Drive/NL2SQL/Models/gazp/exp/sql2nl/r1/best.tar', seed=3, skip_consistency_check=False, skip_intermediate=False, tables='/content/drive/My Drive/NL2SQL/Models/gazp/data/tables.json', warmup=0.1)
/content/drive/My Drive/NL2SQL/Models/

In [ ]:
!python predict.py  --dataset=cosql

In [ ]:
!python predict.py "/content/drive/My Drive/NL2SQL/Models/gazp/exp/sql2nl/r1/best.tar" "/content/drive/My Drive/NL2SQL/Models/gazp/data/dev.json" --tables "/content/drive/My Drive/NL2SQL/Models/gazp/data/tables.json" --db "/content/drive/My Drive/NL2SQL/Models/gazp/data/database" --dcache "/content/drive/My Drive/NL2SQL/Models/gazp/cache/bert" --output "/content/drive/My Drive/NL2SQL/Models/gazp/exp/nl2sql/r1/scores.txt"


2021-06-03 08:55:35.291379: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
^C


In [ ]:
!ls -ltr